# 🤖 Modelo de Recomendación de Películas

**Objetivo:** Implementar algoritmo de Collaborative Filtering Item-Based

**Fecha:** 22 de noviembre de 2024

**Algoritmo:** Item-Based Collaborative Filtering
- Calcula similitud entre películas
- Recomienda películas similares a las que le gustaron al usuario

## Pasos:
1. Cargar datos limpios
2. Crear matriz de similitud
3. Implementar función de recomendación
4. Evaluar el modelo
5. Guardar modelo

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import pickle

# Cargar datos limpios
ratings = pd.read_csv('../data/processed/ratings_clean.csv')
movies = pd.read_csv('../data/processed/movies_clean.csv')

print("✅ Datos cargados")
print(f"📊 Ratings: {len(ratings):,}")
print(f"🎬 Películas: {len(movies):,}")
print(f"👥 Usuarios: {ratings['user_id'].nunique()}")

✅ Datos cargados
📊 Ratings: 94,968
🎬 Películas: 939
👥 Usuarios: 943


In [5]:
# Crear matriz pivote: usuarios en filas, películas en columnas
user_movie_matrix = ratings.pivot_table(
    index='user_id',
    columns='item_id',
    values='rating',
    fill_value=0
)

print("📊 MATRIZ USUARIO-PELÍCULA")
print(f"Dimensiones: {user_movie_matrix.shape}")
print(f"Usuarios: {user_movie_matrix.shape[0]}")
print(f"Películas: {user_movie_matrix.shape[1]}")
print()
print("Primeras 5 filas y 5 columnas:")
user_movie_matrix.iloc[:5, :5]

📊 MATRIZ USUARIO-PELÍCULA
Dimensiones: (943, 939)
Usuarios: 943
Películas: 939

Primeras 5 filas y 5 columnas:


item_id,1,2,3,4,5
user_id,,,,,
1,5.0,3.0,4.0,3.0,3.0
2,4.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0


In [6]:
# Transponer para tener películas en filas
movie_user_matrix = user_movie_matrix.T

# Calcular similitud coseno entre películas
print("⏳ Calculando similitud entre películas...")
movie_similarity = cosine_similarity(movie_user_matrix)

# Convertir a DataFrame para facilitar uso
movie_similarity_df = pd.DataFrame(
    movie_similarity,
    index=movie_user_matrix.index,
    columns=movie_user_matrix.index
)

print("✅ Matriz de similitud calculada")
print(f"Dimensiones: {movie_similarity_df.shape}")
print()
print("Ejemplo - Similitud entre primeras 5 películas:")
movie_similarity_df.iloc[:5, :5]

⏳ Calculando similitud entre películas...
✅ Matriz de similitud calculada
Dimensiones: (939, 939)

Ejemplo - Similitud entre primeras 5 películas:


item_id,1,2,3,4,5
item_id,,,,,
1,1.000000,0.402382,0.330245,0.454938,0.286714
2,0.402382,1.000000,0.273069,0.502571,0.318836
3,0.330245,0.273069,1.000000,0.324866,0.212957
4,0.454938,0.502571,0.324866,1.000000,0.334239
5,0.286714,0.318836,0.212957,0.334239,1.000000


In [8]:
def get_movie_recommendations(movie_id, n_recommendations=10):
    """
    Obtiene recomendaciones basadas en similitud con una película
    
    Args:
        movie_id: ID de la película
        n_recommendations: Número de recomendaciones a retornar
        
    Returns:
        DataFrame con películas recomendadas y su score de similitud
    """
    # Obtener películas similares
    similar_movies = movie_similarity_df[movie_id].sort_values(ascending=False)
    
    # Quitar la misma película
    similar_movies = similar_movies[similar_movies.index != movie_id]
    
    # Top N recomendaciones
    top_movies = similar_movies.head(n_recommendations)
    
    # Crear DataFrame con información de películas
    recommendations = pd.DataFrame({
        'item_id': top_movies.index,
        'similarity_score': top_movies.values
    })
    
    # Agregar títulos
    recommendations = recommendations.merge(
        movies[['item_id', 'title']], 
        on='item_id'
    )
    
    return recommendations[['title', 'similarity_score']]

print("✅ Función de recomendación creada")

✅ Función de recomendación creada


In [9]:
# Probar con una película popular
test_movie_id = 50  # Star Wars (1977)

# Obtener nombre de la película
movie_name = movies[movies['item_id'] == test_movie_id]['title'].values[0]

print(f"🎬 PELÍCULA BASE: {movie_name}")
print(f"📝 ID: {test_movie_id}")
print()
print("🎯 TOP 10 RECOMENDACIONES:")
print("=" * 70)

recommendations = get_movie_recommendations(test_movie_id, n_recommendations=10)

for idx, row in recommendations.iterrows():
    print(f"{idx+1:2d}. {row['title']:50s} - Score: {row['similarity_score']:.3f}")

🎬 PELÍCULA BASE: Star Wars (1977)
📝 ID: 50

🎯 TOP 10 RECOMENDACIONES:
 1. Return of the Jedi (1983)                          - Score: 0.884
 2. Raiders of the Lost Ark (1981)                     - Score: 0.765
 3. Empire Strikes Back, The (1980)                    - Score: 0.750
 4. Toy Story (1995)                                   - Score: 0.735
 5. Godfather, The (1972)                              - Score: 0.697
 6. Independence Day (ID4) (1996)                      - Score: 0.693
 7. Indiana Jones and the Last Crusade (1989)          - Score: 0.689
 8. Fargo (1996)                                       - Score: 0.687
 9. Silence of the Lambs, The (1991)                   - Score: 0.676
10. Star Trek: First Contact (1996)                    - Score: 0.674


In [10]:
# Probar con una película popular
test_movie_id = 50  # Star Wars (1977)

# Obtener nombre de la película
movie_name = movies[movies['item_id'] == test_movie_id]['title'].values[0]

print(f"🎬 PELÍCULA BASE: {movie_name}")
print(f"📝 ID: {test_movie_id}")
print()
print("🎯 TOP 10 RECOMENDACIONES:")
print("=" * 70)

recommendations = get_movie_recommendations(test_movie_id, n_recommendations=10)

for idx, row in recommendations.iterrows():
    print(f"{idx+1:2d}. {row['title']:50s} - Score: {row['similarity_score']:.3f}")

🎬 PELÍCULA BASE: Star Wars (1977)
📝 ID: 50

🎯 TOP 10 RECOMENDACIONES:
 1. Return of the Jedi (1983)                          - Score: 0.884
 2. Raiders of the Lost Ark (1981)                     - Score: 0.765
 3. Empire Strikes Back, The (1980)                    - Score: 0.750
 4. Toy Story (1995)                                   - Score: 0.735
 5. Godfather, The (1972)                              - Score: 0.697
 6. Independence Day (ID4) (1996)                      - Score: 0.693
 7. Indiana Jones and the Last Crusade (1989)          - Score: 0.689
 8. Fargo (1996)                                       - Score: 0.687
 9. Silence of the Lambs, The (1991)                   - Score: 0.676
10. Star Trek: First Contact (1996)                    - Score: 0.674


In [11]:
import os

# Crear carpeta models si no existe
os.makedirs('../src', exist_ok=True)

# Guardar matriz de similitud
with open('../src/movie_similarity.pkl', 'wb') as f:
    pickle.dump(movie_similarity_df, f)

# Guardar datos necesarios
with open('../src/movies_data.pkl', 'wb') as f:
    pickle.dump(movies, f)

print("✅ Modelo guardado en:")
print("   📁 src/movie_similarity.pkl")
print("   📁 src/movies_data.pkl")

✅ Modelo guardado en:
   📁 src/movie_similarity.pkl
   📁 src/movies_data.pkl


In [14]:
print("=" * 70)
print("🎉 MODELO DE RECOMENDACIÓN COMPLETADO")
print("=" * 70)
print()
print("✅ Algoritmo: Item-Based Collaborative Filtering")
print(f"✅ Películas en el modelo: {movie_similarity_df.shape[0]}")

# Mostrar RMSE solo si existe
try:
    print(f"✅ RMSE: {rmse:.3f}")
except NameError:
    print("✅ RMSE: No evaluado (saltado por tiempo)")

print()
print("🎯 PRÓXIMOS PASOS:")
print("   1. Crear interfaz web con Streamlit")
print("   2. Integrar modelo con la interfaz")
print("   3. Permitir que usuarios seleccionen películas")
print("   4. Mostrar recomendaciones personalizadas")
print()
print("=" * 70)

🎉 MODELO DE RECOMENDACIÓN COMPLETADO

✅ Algoritmo: Item-Based Collaborative Filtering
✅ Películas en el modelo: 939
✅ RMSE: No evaluado (saltado por tiempo)

🎯 PRÓXIMOS PASOS:
   1. Crear interfaz web con Streamlit
   2. Integrar modelo con la interfaz
   3. Permitir que usuarios seleccionen películas
   4. Mostrar recomendaciones personalizadas

